In [1]:
import rebound
import numpy as np
import matplotlib.pyplot as plt

We first set up our N-body simulation. Note that we are using the Gragg-Bulirsch-Stoer integrator (BS).

In [7]:
try:
    sim = rebound.Simulation("ss.bin")
except:
    sim = rebound.Simulation()
    sim.add(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])
    sim.save("ss.bin")
sim.integrator = "BS"
#sim.integrator = "WHFast"
sim.dt = sim.particles[1].P/123.

In [8]:
alpha = 8.397 / 1296000 # arcsec/yr to code units (rad/(yr/2pi))
def rhs(ode, yDot, y, t):
    ode.contents.update_particles()

    dp = sim.particles[4] - sim.particles[0]
    x = dp.xyz
    v = dp.vxyz
    L = np.cross(x,v)
    L_hat = L / np.sqrt(np.sum(L**2))
    S = np.array((y[0],y[1],y[2]))
    
    yd = -alpha * np.dot(L_hat,S)* np.cross(L_hat, S)
    yDot[0] = yd[0]
    yDot[1] = yd[1]
    yDot[2] = yd[2]    

In [9]:
ode_ho = sim.create_ode(length=3)

In [10]:
eps = 25.19/180*np.pi # approximate current obliquity in radian
spin0 = np.array((0.,np.sin(eps),np.cos(eps)))
ode_ho.y[0] = spin0[0]
ode_ho.y[1] = spin0[1]
ode_ho.y[2] = spin0[2]
ode_ho.derivatives = rhs

In [ ]:
times = np.linspace(0.,6000.,100)
obliq = np.zeros((len(times)))

for i, t in enumerate(times):
    sim.integrate(t, exact_finish_time=0)
    times[i] = sim.t
    
    dp = sim.particles[4] - sim.particles[0]
    x = dp.xyz
    v = dp.vxyz
    L = np.cross(x,v)
    L_hat = L / np.sqrt(np.sum(L**2))
    spin = [ode_ho.y[0],ode_ho.y[1],ode_ho.y[2]]
    obliq[i] =  np.arccos(np.dot(L_hat,spin))

Let's plot the relative energy error over time for both the N-body and the harmonic oscillator integration.

In [ ]:
fig, ax = plt.subplots(1,1)
precfreq = 7.599 / 1296000
ax.plot(times,obliq*180/np.pi)
ax.plot(times,(obliq[0]+ precfreq*times/41)*180/np.pi)